In [1]:
from pathlib import Path
import os
import sqlite3

import s3fs
import pandas as pd

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


def read_cluster_csv(file_path, endpoint_url='https://storage.budsc.midwest-datascience.com'):
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    return pd.read_csv(s3.open(file_path, mode='rb'))

## Create and Load Measurements Table

In [2]:
def create_measurements_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS measurements (
        visit_id integer NOT NULL,
        person_id text NOT NULL,
        quantity text,
        reading real,
        FOREIGN KEY (visit_id) REFERENCES visits (visit_id),
        FOREIGN KEY (person_id) REFERENCES people (person_id)
        );
    """

    c = conn.cursor()
    c.execute(sql)
    
def load_measurements_table(conn):
    create_measurements_table(conn)
    df = pd.read_json('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment02/results/kvdb/measurements.json')
    print('\n')
    print(df)
    measurements = df.values
    c = conn.cursor()
    c.execute('DELETE FROM measurements;') # Delete data if exists
    c.executemany('INSERT INTO measurements VALUES (?,?,?,?)', measurements)

## Create and Load People Table

In [3]:
def create_people_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS people (
        person_id text PRIMARY KEY,
        personal_name text NOT NULL,
        family_name text NOT NULL
        );
    """
    ## TODO: Complete SQL
    c = conn.cursor()
    c.execute(sql)
    
def load_people_table(conn):
    create_people_table(conn)
    ## TODO: Complete code
    df = pd.read_json('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment02/results/kvdb/people.json').T
    print('\n')
    print(df)
    people = df.values
    c = conn.cursor()
    c.execute('DELETE FROM people;') # Delete data if exists
    c.executemany('INSERT INTO people VALUES (?,?,?)', people)

## Create and Load Sites Table

In [4]:
def create_sites_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id text PRIMARY KEY,
        latitude double NOT NULL,
        longitude double NOT NULL
        );
    """

    c = conn.cursor()
    c.execute(sql)

def load_sites_table(conn):
    create_sites_table(conn)
    ## TODO: Complete code
    df = pd.read_json('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment02/results/kvdb/sites.json').T
    print('\n')
    print(df)
    sites = df.values
    c = conn.cursor()
    c.execute('DELETE FROM sites;') # Delete data if exists
    c.executemany('INSERT INTO sites VALUES (?,?,?)', sites)

## Create and Load Visits Table

In [5]:
def create_visits_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS visits (
        visit_id integer PRIMARY KEY,
        site_id text NOT NULL,
        visit_date text,
        FOREIGN KEY (site_id) REFERENCES sites (site_id)
        );
    """

    c = conn.cursor()
    c.execute(sql)

def load_visits_table(conn):
    create_visits_table(conn)
    ## TODO: Complete code
    # this one has an issue with nan values
    # read it from file into a string

    import json

    text=open('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment02/results/kvdb/visited.json','r')
    x=text.read()

    y=json.loads(x)
    df=pd.DataFrame(y)
    df = df.T
    print('\n')
    print(df.head())
    visits = df.values
    c = conn.cursor()
    c.execute('DELETE FROM visits;') # Delete data if exists
    c.executemany('INSERT INTO visits VALUES (?,?,?)', visits)

## Create DB and Load Tables

In [6]:
db_path = results_dir.joinpath('patient-info.db')
conn = sqlite3.connect(str(db_path))
# TODO: Uncomment once functions completed
load_people_table(conn)
load_sites_table(conn)
load_visits_table(conn)
load_measurements_table(conn)

conn.commit()
conn.close()
print('\nTables built successfully')



         person_id personal_name family_name
danforth  danforth         Frank    Danforth
dyer          dyer       William        Dyer
lake          lake      Anderson        Lake
pb              pb         Frank     Pabodie
roe            roe     Valentina     Roerich


      site_id latitude longitude
DR-1     DR-1   -49.85   -128.57
DR-3     DR-3   -47.15   -126.72
MSK-4   MSK-4   -48.87    -123.4


    visit_id site_id  visit_date
619      619    DR-1  1927-02-08
622      622    DR-1  1927-02-10
734      734    DR-3  1930-01-07
735      735    DR-3  1930-01-12
751      751    DR-3  1930-02-26


    visit_id person_id quantity  reading
0        619      dyer      rad     9.82
1        619      dyer      sal     0.13
2        622      dyer      rad     7.80
3        622      dyer      sal     0.09
4        734        pb      rad     8.41
5        734      lake      sal     0.05
6        734        pb     temp   -21.50
7        735        pb      rad     7.22
8        735        pb 